In this piece of code I simultaneously optimize over the unitary basis rotation and the mps tensors.

This is often done in "DMRG-SCF", but people conventionally optimize the matrix product state quite far before taking a small step in the orbital rotations. In this notebook I try to combine grassmann gradient descent with "DMRG-SCF", where I optimize the orbital basis after every linesearch. This may be more efficient...

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental
using GaussianBasis,DelimitedFiles

LinearAlgebra.BLAS.set_num_threads(1);
#TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();

In [2]:
using Base.Threads; nthreads()

12

In [3]:
bset = BasisSet("6-31g", """
N        0.00      0.00     0.00      
N        2.118      0.00     0.00""")

6-31g Basis Set
Type: Spherical   Backend: Libcint

Number of shells: 10
Number of basis:  18

N: 1s 2s 1p 3s 2p 
N: 1s 2s 1p 3s 2p

I will keep the first 3 orbitals frozen filled, treat the next 10 orbitals using MPS, and keep the rest frozen free.

In [4]:
N_ORBS = bset.nbas;
N_frozen_filled = 4;
N_active = 12;
N_frozen_empty = N_ORBS-N_frozen_filled-N_active;
NELEC = 14;
MS2 = 0;

The state is now smaller:

In [8]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-(NELEC-N_frozen_filled*2),MS2//2,mod1(-(NELEC-N_frozen_filled*2),2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -(NELEC-N_frozen_filled*2):0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,Float64,N_active,psp,virtual;left,right);

The code works through a weird new type. We cannot work directly with an MPO hamiltonian, because I want to consider general unitary rotations between frozen orbitals and the active space. The MPO will only describe the active space, and therefore does not contain enough information.

In [9]:
nh = CASSCF_Ham(bset,N_frozen_filled+1:N_frozen_filled+N_active);

Let's try it out!

In [10]:
# first some dmrg to ramp up the bond dimension
mpo_opp = MPSKitExperimental.mpo_representation(nh)[1]; # will be made easier in future
(st,env) = find_groundstate(st,mpo_opp,DMRG2(maxiter=1,trscheme=truncdim(100)),disk_environments(st,mpo_opp));

# grassmannscf simultaneously converges the state and the oribtals
@time (st,nh) = find_groundstate(st,nh,GrassmannSCF(maxiter=200));

┌ Info: Iteraton 0 error 0.9980757676863208
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/vdKxJ/src/algorithms/groundstate/dmrg.jl:112


┌ Info: CG: initializing with f = -90.386366875451, ‖∇f‖ = 3.6926e-01
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:39


┌ Info: CG: initializing with f = -90.389018624928, ‖∇f‖ = 1.9354e+01
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:39


┌ Info: CG: iter    1: f = -97.954144995721, ‖∇f‖ = 2.2007e+00, α = 8.25e-01, β = 0.00e+00, nfg = 2
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:95


┌ Info: CG: iter    2: f = -98.080035382825, ‖∇f‖ = 2.4473e+00, α = 7.29e-02, β = 2.88e-01, nfg = 4
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:95


...

┌ Info: CG: iter  199: f = -108.528273663332, ‖∇f‖ = 2.4593e-01, α = 7.14e-02, β = 1.57e+00, nfg = 2
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:95


┌ Info: CG: initializing with f = -108.531490724976, ‖∇f‖ = 3.2469e-02
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:39


┌ Info: CG: converged after 1 iterations: f = -108.534059861870, ‖∇f‖ = 1.0437e-02
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:112


13801.746204 seconds (173.65 G allocations: 12.839 TiB, 12.19% gc time, 0.03% compilation time: 40% of which was recompilation)


┌ Warning: CG: not converged to requested tol: f = -108.534059861870, ‖∇f‖ = 2.5353e-01
└ @ OptimKit /home/maavdamm/OptimKit.jl/src/cg.jl:115


In practice, one should probably start from a well informed initial guess, instead of using atomic orbitals. Nevertheless, as a proof of principle, it works.